# Lab 1: "Quantization and Entropy Coding"

* Student full name:
* Student email:

# 0. Introduction
During this lab, you will work with the concepts seen during the Lectures. The lab is divided into excercises related to the following fundamental concepts:

- Uniform quantization, and quantization error
- Entropy
- Huffman Coding / Decoding

The delivery will consist of a copy of this notebook, with your code, figures, and answers. Please, once finished, run all cells (hopefully without errors) and download your notebook as a `.ipynb` file, which will be submitted to Aula Global.

We recommend to work directly on a Google Colab notebook, since we do not require a powerful server, and everything is already set up. In any case, you can work in your own laptop if you prefer to, by installing all required dependencies.

In [ ]:
# Install a few dependencies that are not included by default
!apt-get install -y ffmpeg
!ffmpeg -y -i https://www.voiptroubleshooter.com/open_speech/american/OSR_us_000_0010_8k.wav -af "volume=9dB" -t 20 -ac 1 -ar 11025 -acodec pcm_s16le speech.wav
!pip install huffman
!pip install bitarray

In [ ]:
# Check we are using python 3
import sys
assert(sys.version_info.major == 3)

In [ ]:
# Some useful functions that will be used along the assignment

import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio

def generate_decay_signal(sample_rate=11025, freq=1000, phase=0, duration=20):
    """ Generate a tone modulated by a decay signal

    This signal is useful to study the impact of quantization, since it uses
    the full dynamic range.

    Args:
        sample_rate (int): Sample rate
        freq (float): Tone frequency
        phase (float): Tone phase
        duration (float): Signal duration in seconds

    Returns:
        (np.array) Samples array
    """
    times = np.arange(0, duration, 1/sample_rate)
    cy = np.cos(2 * np.pi * times * freq + phase)
    sy = np.sin(2 * np.pi * times * freq + phase)
    pulse = cy + sy
    unclipped_decay_signal = pulse * 25 * np.exp(-times * 0.5)
    decay_signal = np.minimum(np.maximum(unclipped_decay_signal, -1), 1)
    return decay_signal

def display_spectrogram(x, hop_length=128, n_fft=256, sample_rate=11025):
    """ Display spectrogram of signal x

    Args:
        x (numpy.array): Array of samples
        hop_length (int): Spectrogram hop length
        n_fft (int): Spectrogram fft size
        sample_rate (int): Sample rate
    """
    from librosa.display import specshow
    data = librosa.amplitude_to_db(
        np.abs(librosa.core.stft(x,
                                 hop_length=hop_length,
                                 n_fft=n_fft,
                                 center=True)),
                                 ref=np.max)
    plt.figure(figsize=(10, 5))
    specshow(data, x_axis='time', y_axis='linear', cmap='inferno',
            sr=sample_rate, hop_length=hop_length)
    plt.colorbar(format='%+2.0f dB')

def display_audio_player(x, sample_rate=11025):
    """ Display an embedded audio player

    Args:
        x (numpy.array): Array of samples
        sample_rate (int): Sample rate
    """
    return Audio(x, rate=sample_rate)

# 1. Quantization (40%)

## Exercise 1.1 (10%)

**a) Write a function to plot the waveform of an audio signal (add documentation as in `display_spectrogram`, use `matplotlib`, add at least axis names, make it nice)**

**b) Plot the waveform of the signal generated with `generate_decay_signal()`**

**c) Embed an audio player in order to listen to the signal**

**d) Display the spectrogram using `display_spectrogram` function**

**e) Observe the waveform, the spectrogram, and listen to the audio. Describe briefly your observations**

(Answer here)

## Exercise 1.2 (25%)

Before describing the task, we present an explanation that might help to better understand the approach of the exercise.

### About symmetrical mid-rise vs. asymmetrical mid-tread

In the bibliography suggested:

*Introduction to Digital Audio Coding and Standards. Authors: Bosi, Marina and Goldberg, Richard E.*
http://last.hit.bme.hu/download/vidtechlab/fcc/literature/audio/audio_coding_standards_book.pdf

The following claim is made:

*We typically choose quantizers that are symmetric in that there are an equal number of levels (codes) for positive and negative numbers.*

However, in the case of standard PCM audio codings, this is not the case. For instance, 16 bits PCM WAV coding (16 bits signed integers) assigns `-32768` to $-1.0$ amplitude value, `0` to $0.0$ amplitude value, and `+32767` to $+1.0 - \frac{1}{2^{15}}$ amplitude value. The amplitude value $1.0$ has no representation in 16 bits PCM WAV audio files.

In [ ]:
# Some code to illustrate the previous explanation.

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html
from scipy.io.wavfile import write
from scipy.io.wavfile import read

x = np.array([-32768, -1, 0, 1, 32767], dtype=np.int16)
write('codec__pcm_s16le.wav', 8000, x)
sr, y = read('codec__pcm_s16le.wav')

In [ ]:
!ffprobe  -v error -i codec__pcm_s16le.wav -show_entries stream=codec_name

[STREAM]
codec_name=pcm_s16le
[/STREAM]


In [ ]:
# Convert to float 32 bits codec
!ffmpeg -y -v error -i codec__pcm_s16le.wav -acodec pcm_f32le codec__pcm_f32le.wav

In [ ]:
sr, z = read('codec__pcm_f32le.wav')
for sample in z:
    print(sample)

-1.0
-3.0517578e-05
0.0
3.0517578e-05
0.9999695




---

**a) The following function "simulates" the quantization of a signal using an asymmetric uniform mid-tread quantizer with N bits. Please, explain how the function works, and why each line is needed. Imagine you select N=2: does it mean this python program is using 2 bits to store internally each of the numbers produced by this `quantize(x, N)` python function? (you can check it with https://numpy.org/doc/stable/reference/generated/numpy.ndarray.nbytes.html). Why do we say this function "simulates" the quantization process?**

```python
def quantize(x, N):
    """ Quantize signal using N bits
    Args:
        x (numpy.array): Array of samples
        N (int): Number of bits
    """
    x = np.array(x)
    quantized = np.round(x * 2**(N-1))
    quantized = np.minimum(quantized, 2**(N-1) - 1)
    quantized = np.maximum(quantized, -2**(N - 1))
    quantized = quantized.astype(np.int16)
    return quantized.reshape(x.shape)
```

**Remember this mid-tread quantization function:**

![](https://vru.vibrationresearch.com/wp-content/uploads/2019/10/quantization_midtread_8bit.png-1.png)

**A set of tests are provided to check if your function is correctly implemented or not:**

In [ ]:
# Run this cell for automatic testing

def dequantize(codes, N):
    """ Dequantize signal using N bits

    Args:
        x (numpy.array): Array of samples
        N (int): Number of bits
    """
    return np.array(codes) / 2**(N - 1)

# Test
test_vector = [-1.5,
               -1.0,
               -0.88,
               -0.86,
               -0.126,
               -0.125,
               0,
               0.125,
               0.126,
               1.0,
               1.5]
expected_codes = [-4, -4, -4, -3, -1, 0, 0, 0, 1, 3, 3]
expected_dequantized = [-1.0,
                        -1.0,
                        -1.0,
                        -0.75,
                        -0.25,
                        0.0,
                        0.0,
                        0.0,
                        0.25,
                        0.75,
                        0.75]
codes = quantize(test_vector, 3)
dequantized = dequantize(codes, 3)

for x, y, z, e_y, e_z in zip(test_vector, codes, dequantized,
                             expected_codes, expected_dequantized):
    print("x: {0}, code: {1}, dequant: {2}, expected code: {3}, expected dequant: {4}".format(x, y, z, e_y, e_z))
    assert(y == e_y)
    assert(z == e_z)

# Quantization function using 3 bits
plt.figure()
x = np.arange(-1.0, 1.0, 0.001)
y = dequantize(quantize(x, 3), 3)
plt.plot(x, y)
plt.grid()
plt.title('3 bits quantization function (Δ=0.25)')
plt.show()

**b) Complete the function `get_snr(N)` to compute the SNR in dBs given a number of bits for a signal with uniform distribution.**

**Note that we added a hack (adding or subtracting $\frac{1}{2^N}$) in order to use a mid-rise quantizer because the formula $SNR = N \times 6.021$ is valid for a symmetric quantizer with $2^N$ steps.**

Here, please remember that the SNR in dBs is given by:

snr = 10 * np.log10(energy_x / energy_qn)

Where energy_x is the energy of signal x, and energy_qn is the energy of the quantization error (i.e. x-y).

Do not forget to return the snr at the end of the function.

In [ ]:
def get_snr_uniform(N):
    x = np.random.uniform(low=-1.0, high=1.0, size=(1000000,))
    y = dequantize(quantize(x - (1/2**N), N), N) + 1/2**N  # for this experiment
                                                           # we use a mid-rise quantizer
                                                           # (symmetry reasons)
    # Complete this function:

def get_theoretical_snr_uniform(N):
    theo_snr = N * 6.021
    return theo_snr


N_range = range(1, 16)
snrs = [get_snr_uniform(N) for N in N_range]
theo_snrs = [get_theoretical_snr_uniform(N) for N in N_range]
for (N, snr, theo_snr) in zip(N_range, snrs, theo_snrs):
    print("N:{0} bits - SNR:{1:.2f}dB - Theoretical SNR:{2:.2f}dB".format(N, snr, theo_snr))

## Exercise 1.3 (5%)

**a) Create a decay signal using `generate_decay_signal()` and apply a quantization + dequantization process to the previous signal using 4 bits. Then display the waveform, the spectrogram, and an audio player of the resulting signal.**

Tip: In order to display the waveform, spectrogram, or audio player, the signal must be in standard range (float from -1.0 to 1.0). That is why, we suggest to first apply a quantization (in order to convert from a float range to N discrete steps), and then a dequantization (for going back from N discrete steps to a float range between -1.0 and 1.0).

**b) Apply a quantization + dequantization process to the previous signal using 8 bits, and display the waveform, the spectrogram, and an audio player of the resulting signal.**

**c) Include any observation you might find interesting about this analysis**

(Your observations here)

# 2. Entropy calculation (20%)

## Exercise 2.1 (20%)
**a) Develop a function that allows you to plot the entropy of a source with variable output probabilities. We wish to observe that the maximum source entropy does indeed occur when the source outputs are equally likely.**

**The head of the function should be:**

```python
def calculate_entropy(probs):
    """
        Args:
            probs (np.array): Array of probs. for each symbol in the source
        Returns:
            (float) H (i.e. Entropy)
    """
```

**Please, take into account the cases where probabilities are 0.0, or incorrect inputs with negative probabilities (we can return `np.nan` in that case)**

First assert is the most important one:

```python
assert(calculate_entropy([0.5, 0.5]) == 1.0)
```

Complying with the rest of asserts is a bit harder:

```python
assert(calculate_entropy([1.0, 0.0]) == 0.0)
assert(np.isnan(calculate_entropy([-1.0, 0.0])))
assert(calculate_entropy([0.0, 1.0]) == 0.0)
```

So I consider the three last ones as optional (they just consider edge cases or incorrect inputs).

In [ ]:
assert(calculate_entropy([0.5, 0.5]) == 1.0)
assert(calculate_entropy([1.0, 0.0]) == 0.0)
assert(np.isnan(calculate_entropy([-1.0, 0.0])))
assert(calculate_entropy([0.0, 1.0]) == 0.0)

**b) Consider a simple two-output source $[m_1, m_2]$ with respective probabilities $[a, 1 - a]$, and plot the entropy as a function of the parameter $a$, where $0 ≤ a ≤ 1$.**


**c) Optional: consider more complex cases such as a three output source $[m_1, m_2, m_3]$ with
respective probabilities $[a, b, 1 − a − b]$. Plot a 2D image with `plt.imshow` where entropy is displayed as a function of `a` and `b` parameters.**

# 3. Huffman Encoding (40%)

In this exercise, Huffman coding will be applied to compress `speech.wav` file.


In [ ]:
x, sr = librosa.load("speech.wav",
                     sr=11025, mono=True)
display_waveform(x)
display_spectrogram(x)
display_audio_player(x)

## Exercise 3.1 (15%)

**a) Quantize signal `x` using 4 bits. Call it `y`**

Tip: The answer is:

```
bits_per_sample = 4
y = quantize(x, bits_per_sample)
```

Please, do not spend time dealing with it.

**b) Implement a function that computes the probability of occurrence per symbol for `y`.**

**The header of the function must be:**

Tip: With `np.unique(y, return_counts=True)`, the answer can be done with three lines. Beware that np.unique returns counts, instead of probabilities, so you have to normalize them (probabilities sum 1.0).

Remember that a function can return more than one element in python, e.g.

```python
def get_prob_per_symbol(y):
    """ Get symbol occurrence probability
    Args:
        x (np.array): Input signal
    Returns:
        (np.array, np.array): (sorted symbols, corresponding probabilities)
    """
    # Complete function here
    return symbols, probs
symbols, probs = get_prob_per_symbol(y)
```


**c) Use the implemented function to display the symbol probability distribution for `y`**

**d) Do you think this probability distribution will be similar in other speech signals?**

(Answer here)

**e) Compute the entropy of `y`**

## Exercise 3.2 (15%)

**a) Review the documentation of `huffman` python package:**

https://pypi.org/project/huffman/

**and generate a codebook for the previously computed symbols and probabilities.**

Given a list of symbols, and a list of probabilities, (as returned by get_prob_per_symbol)
the input to huffman.codebook(symbol_probs) must be:

```symbol_probs = [(s, p) for (s, p) in zip(symbols, probs)]```

It is the format huffman package understands.


**b) Consider the bit array `10010110001`: is the associated symbols sequence unique?**

(Answer here)

**c) Compute:**
- **Code mean length**
- **Code efficiency**
- **Compression ratio**


In [ ]:
print("Mean length: {0}".format(mean_length))
print("Code efficiency: {0:.2f}%".format(100 * code_efficiency))
print("Compression ratio: {0:.2f}%".format(100 * compression_ratio))

## Exercise 3.3 (10%)

**a) Using `bitarray` python package, encode the signal into a sequence of bits (method `.encode()` of `bitarray` package) with the codebook previously generated.**

https://pypi.org/project/bitarray/

Update:
The answer is:

```
from bitarray import bitarray
codebook_bitarray = {}
for (k, v) in codebook.items():  # codebook as returned by huffman package
  codebook_bitarray[k] = bitarray(v)
a = bitarray()
a.encode(codebook_bitarray, y)
print(a)
```

Instead of implementing it from scratch, please explain what it does.

**b) Decode the signal using `.decode()` method of `bitarray` package, and verify it equals `y` for all samples.**

**c) Compute the size in kilobits per second of the bit arrays before (i.e. fix-length) and after Huffman encoding.**

In [ ]:
print("kbps fixed-length: {0}kbps".format(kbps_per_second_fixlength))
print("kbps Huffman: {0}kbps".format(kbps_per_second_encoded))

**d) Are the previous kbps values consistent with the compression ratio already computed before?**

(Answer here)